In [1]:
import riskfolio as rp
import pandas as pd
import numpy as np
import yfinance as yf
import warnings
import xlwings as xw
import os
import matplotlib.pyplot as plt


In [2]:
# Function to calculate portfolio performance and save reports using Riskfolio-Lib plots

# Create the directory for reports if it doesn't exist
report_dir = "../reports/05 - portfolio_optimization_riskfolio"
os.makedirs(report_dir, exist_ok=True)


# Example of running the repor

In [3]:

warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.4%}'.format

# Load S&P 500 ticker symbols
#sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
tickers = ['SMCI', 'NVDA', 'HRB', 'CL', 'XPO', 'RCL', 'HD', 'EXAS', 'BLDR', 'FTNT',
           'ORCL', 'GE', 'TPX', 'CCL', 'LII', 'SAIA', 'FTI', 'PHM', 'META', 'CELH',
           'LRCX', 'SPB', 'JBL', 'AVGO', 'ADBE', 'VRSK', 'SSD', 'LSCC', 'EME',
           'TOL', 'CXT', 'HUBS', 'APO', 'AMAT', 'MSI', 'MA', 'FIX', 'SCCO', 'ALSN',
           'KLAC', 'NFLX', 'CAT', 'OC', 'PVH', 'LLY', 'AXON', 'EXP', 'LNW', 'URI',
           'KMB']

# Download historical price data
data = yf.download(tickers, start='2024-01-01', end='2024-09-30', interval='1h')['Adj Close']
data = data.tz_localize(None)

# Forward-fill missing values and drop any columns that still have NaNs
data = data.ffill().dropna(axis=1)

# Calculate daily returns from adjusted closing prices
returns = data.pct_change().dropna()

# Display the first few rows of returns to verify the data is ready for portfolio optimization
returns.head()


[*********************100%***********************]  50 of 50 completed


Ticker,ADBE,ALSN,AMAT,APO,AVGO,AXON,BLDR,CAT,CCL,CELH,...,SAIA,SCCO,SMCI,SPB,SSD,TOL,TPX,URI,VRSK,XPO
Datetime,,,,,,,,,,,,,,,,,,,,,
2024-01-02 15:30:00,0.0190%,-1.3260%,0.3647%,-0.0793%,-0.1135%,0.5256%,-0.0723%,-0.5253%,-1.3955%,3.1046%,...,-1.2443%,-0.0235%,1.0932%,-0.0811%,-0.7086%,-0.6714%,-0.4184%,-0.5990%,-0.2494%,0.0000%
2024-01-02 16:30:00,-0.2034%,0.1309%,-0.1785%,0.5367%,-0.1611%,0.1782%,-0.0784%,-0.0409%,0.4246%,0.2080%,...,-0.4827%,-0.2630%,0.0246%,0.3373%,-0.1603%,0.0641%,0.1100%,0.2029%,-0.1288%,-0.3410%
2024-01-02 17:30:00,0.0078%,-0.3050%,-0.1501%,-0.1961%,-0.2533%,-0.4965%,-0.1749%,0.1465%,-0.4515%,0.0173%,...,-0.3582%,-0.3193%,0.1860%,0.4357%,-0.2797%,-0.1972%,-0.3398%,0.0581%,-0.0969%,-0.3775%
2024-01-02 18:30:00,0.3100%,-0.3759%,-0.3614%,-0.2947%,-0.3580%,-0.4308%,-0.3444%,-0.3369%,-1.4434%,0.7096%,...,-0.1049%,-0.1029%,-0.2925%,-0.0248%,-0.0987%,-0.4249%,0.0501%,-0.3168%,0.2384%,0.1776%
2024-01-02 19:30:00,-0.4476%,-0.2896%,-1.3287%,-0.2737%,-0.5203%,-0.1297%,-0.3275%,-0.4678%,-0.3735%,0.3093%,...,0.8625%,-0.9415%,-0.4129%,-0.6385%,0.1612%,-0.2977%,-0.5312%,-0.4873%,-0.2041%,-0.1891%


## Markowitz Mean-Variance Analysis Framework

### Introduction

The **Mean-Variance Analysis**, introduced by Harry Markowitz in 1952, is the foundation of modern portfolio theory (MPT). This framework provides a method for constructing a portfolio that maximizes expected return for a given level of risk or minimizes risk for a given level of expected return. It is based on two key parameters of asset returns:
1. **Expected Return (Mean)**: The anticipated average return of an asset or portfolio.
2. **Risk (Variance or Standard Deviation)**: The degree of variation in asset returns, measured by the variance or its square root, the standard deviation.

Markowitz's model assumes that investors are risk-averse, meaning they prefer less risk for the same level of expected return. As a result, the Mean-Variance Optimization aims to find the portfolio that provides the best possible trade-off between risk and return.

### Key Components

1. **Expected Return**: The expected return of a portfolio is the weighted sum of the expected returns of the individual assets. Mathematically:
    $$
    E(R_p) = \sum_{i=1}^{n} w_i \cdot E(R_i)
    $$
    
-   Where:
    - $E(R_p)$ = Expected return of the portfolio
    - $w_i$ = Weight of asset $i$ in the portfolio
    - $E(R_i)$ = Expected return of asset $i$

2. **Portfolio Variance (Risk)**: The portfolio variance captures the total risk of the portfolio, accounting for the variances of individual assets and the covariances between them. It is calculated as:
$$
\text{Var}(R_p) = \sum_{i=1}^{n} \sum_{j=1}^{n} w_i \cdot w_j \cdot \text{Cov}(R_i, R_j)
$$
-   Where:
    - $\text{Var}(R_p)$ = Variance of the portfolio
    - $w_i, w_j$ = Weights of assets $i$ and $j$
    - $\text{Cov}(R_i, R_j)$ = Covariance between the returns of assets $i$ and $j$
- The **standard deviation** is the square root of the variance and is commonly used as a measure of risk.

3. **Covariance Matrix**:
The covariance matrix plays a central role in determining portfolio risk. It measures how asset returns move together (correlate). Positive covariance implies that two assets tend to move in the same direction, while negative covariance implies that they move in opposite directions.
   
4. **Efficient Frontier**:
The efficient frontier is a curve that represents the set of optimal portfolios that offer the maximum expected return for a given level of risk or the minimum risk for a given expected return. Portfolios on this frontier are considered "efficient."
   
5. **Global Minimum Variance Portfolio**:
This is the portfolio on the efficient frontier that has the lowest possible risk (variance). It is often used by very risk-averse investors.
   
6. **Capital Allocation Line (CAL)**:
When we include a risk-free asset (like government bonds), the Capital Allocation Line represents the risk-return combinations of portfolios formed by blending the risk-free asset and risky portfolios. The point where the CAL is tangent to the efficient frontier gives the **tangency portfolio**, which has the highest Sharpe ratio (risk-adjusted return).

### Assumptions of Mean-Variance Optimization
1. **Risk and Return are normally distributed**: The model assumes that asset returns follow a normal distribution, which allows risk to be summarized using variance.
2. **Investors are rational and risk-averse**: Investors prefer less risk for a given level of expected return.
3. **Investors make decisions based on expected return and variance only**: Other factors, such as liquidity or transaction costs, are not considered.
4. **Markets are efficient**: Asset prices reflect all available information.

### Optimization Problem

The Mean-Variance Optimization involves solving a quadratic programming problem where the objective is to minimize portfolio variance, subject to constraints on the expected return and portfolio weights. The general optimization problem can be formulated as:

$$
\min_{w} \quad w^\top \Sigma w
$$
$$
\text{subject to} \quad w^\top \mu = E(R_p), \quad \sum w_i = 1, \quad w_i \geq 0 \, \forall i
$$

Where:
- $w$ = Portfolio weights vector.
- $\Sigma$ = Covariance matrix of asset returns.
- $\mu$ = Vector of expected returns of assets.
- $E(R_p)$ = Expected return of the portfolio.

The constraint $\sum w_i = 1$ ensures that the portfolio weights sum to 100%, and $w_i \geq 0$ ensures that short selling (negative weights) is not allowed.

### Benefits of Mean-Variance Optimization
- Provides a structured approach to portfolio selection.
- Helps investors achieve an optimal trade-off between risk and return.
- Supports diversification by explicitly accounting for asset correlations (via the covariance matrix).

### Limitations of Mean-Variance Optimization
- **Estimation Errors**: The optimization is highly sensitive to the inputs (expected returns and covariances). Small changes in these estimates can result in significantly different portfolio weights.
- **Assumption of Normality**: Asset returns may not always follow a normal distribution, especially in the case of fat tails or skewed distributions.
- **Ignores Higher Moments**: The model focuses only on the first two moments (mean and variance), neglecting skewness and kurtosis.
- **Doesn't Consider Transaction Costs**: The model doesn't account for real-world costs such as transaction fees or taxes when rebalancing the portfolio.

### Conclusion

Markowitz’s Mean-Variance framework is a powerful tool for portfolio optimization, and it laid the foundation for modern portfolio theory. However, while it offers valuable insights into the risk-return trade-off, its real-world application requires careful handling of input estimates and may need to be complemented with more advanced models that consider factors beyond mean and variance.



In [4]:
# Initialize a dictionary to store portfolio weights for each optimization method
portfolio_weights = {}

# Create the directory for reports if it doesn't exist
report_dir = "../reports/05 - portfolio_optimization_riskfolio"
os.makedirs(report_dir, exist_ok=True)


In [5]:
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Main optimization function with debugging and error handling
def portfolio_optimization(pf_returns, method_num, method_name, model, opt_method, **kwargs):
    method_id = f"{method_num:02d} - {method_name}"
    file_path = os.path.join("../reports/05 - portfolio_optimization_riskfolio", method_id)

    logging.info(f"Running {method_name} Portfolio Optimization using {model} model and {opt_method} method")

    try:
        # Create Portfolio object
        if model in ['Classic', 'BL', 'FM', 'BLFM']:
            port = rp.Portfolio(returns=pf_returns)
        elif model in ['HRP', 'HERC']:
            port = rp.HCPortfolio(returns=pf_returns)
        logging.info("Portfolio object created successfully")

        # Get the asset stats based on the specified method for expected returns and covariance
        method_mu = kwargs.get('method_mu', 'hist')  # Default is 'hist'
        method_cov = kwargs.get('method_cov', 'hist')  # Default is 'hist'
        port.assets_stats(method_mu=method_mu, method_cov=method_cov)
        logging.info("Asset stats calculated with method_mu='%s' and method_cov='%s'", method_mu, method_cov)

    except Exception as e:
        logging.error("Error in creating portfolio object or calculating asset stats: %s", e)
        return None  # Exit early if something goes wrong here

    try:
        # Optimization parameters
        model = kwargs.get('model', 'Classic')  # Default model is 'Classic'
        rm = kwargs.get('rm', 'MV')  # Default risk measure is 'MV'
        obj = kwargs.get('obj', 'Sharpe')  # Default objective is 'Sharpe'
        rf = kwargs.get('rf', 0)  # Default risk-free rate is 0
        l = kwargs.get('l', 0)  # Risk aversion factor, useful for Utility maximization
        hist = kwargs.get('hist', True)  # Use historical scenarios by default

        # Select optimization method
        logging.info(f"Optimization method selected: {opt_method}")
        if opt_method == 'optimization':
            w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)
        elif opt_method == 'rp_optimization':
            w = port.rp_optimization(model=model, rm=rm, rf=rf, hist=hist)
        elif opt_method == 'rrp_optimization':
            w = port.rrp_optimization(model=model, rm=rm, rf=rf, hist=hist)
        elif opt_method == 'wc_optimization':
            w = port.wc_optimization(model=model, rm=rm, rf=rf, hist=hist)
        elif opt_method == 'owa_optimization':
            w = port.owa_optimization(model=model, rm=rm, rf=rf, hist=hist)

        logging.info("Optimization completed successfully")

    except Exception as e:
        logging.error("Error during optimization: %s", e)
        return None  # Exit early if something goes wrong during optimization

    try:
        # Store the portfolio weights
        portfolio_weights = {method_name: w}  # Store the weights for later use
        logging.info("Portfolio weights stored successfully")

        # Generate the efficient frontier
        points = kwargs.get('points', 50)  # Number of points on the efficient frontier
        frontier = port.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)
        logging.info(f"Efficient frontier generated with {points} points")

    except Exception as e:
        logging.error("Error generating efficient frontier or storing portfolio weights: %s", e)
        return None  # Exit early if efficient frontier calculation fails

    try:
        # Riskfolio plotting and reporting
        label = kwargs.get('label', 'Max Risk Adjusted Return Portfolio')
        mu = port.mu  # Expected returns
        cov = port.cov  # Covariance matrix
        returns = port.returns  # Returns of the assets

        # Generate Excel report
        rp.excel_report(returns, w, rf=rf, alpha=0.05, t_factor=252, ini_days=1, days_per_year=252, name=file_path)
        logging.info("Excel report generated successfully at %s", file_path)

        # Create additional plots
        fig, ax = plt.subplots(figsize=(10, 10))
        ax = rp.plot_drawdown(returns, w, alpha=0.05, height=10, width=10, solver='CLARABEL', ax=ax)
        logging.info("Drawdown plot created successfully")

        fig_1, ax_1 = plt.subplots(figsize=(10, 5))
        ax_1 = rp.plot_pie(w=w, title='Portfolio Composition', others=0.05, nrow=25, cmap="tab20", height=5, width=10, ax=ax_1)

        fig_2, ax_2 = plt.subplots(figsize=(10, 5))
        ax_2 = rp.plot_risk_con(w=w, cov=cov, returns=returns, rm=rm, rf=rf, alpha=0.05, color="tab:blue", height=5, width=10, t_factor=252, ax=ax_2)

        fig_3, ax_3 = plt.subplots(figsize=(10, 5))
        ax_3 = rp.plot_bar(w, title='Portfolio Weights', kind="v", others=0.05, nrow=25, height=5, width=10, ax=ax_3)

        fig_4, ax_4 = plt.subplots(figsize=(10, 5))
        ax_4 = rp.plot_hist(returns, w, alpha=0.05, bins=50, height=5, width=10, ax=ax_4)

        fig_5, ax_5 = plt.subplots(figsize=(10, 5))
        ax_5 = rp.plot_frontier(w_frontier=frontier, mu=mu, cov=cov, returns=returns, rm=rm, rf=rf, alpha=0.01, cmap='viridis', w=w, label=label, marker='*', s=16, c='r', height=5, width=10, ax=ax_5)

        logging.info("All plots created successfully")

    except Exception as e:
        logging.error("Error during plotting or reporting: %s", e)
        return None  # Exit early if plotting or reporting fails

    try:
        # Save plots in Excel using xlwings
        with xw.App(visible=False) as app:
            wb = xw.Book(f"{file_path}.xlsx")
            sheet1 = wb.sheets[0]

            # Insert the plots into the Excel sheet
            sheet1.pictures.add(fig, name="Drawdown Plot", update=True, top=sheet1.range("O1").top, left=sheet1.range("O1").left)
            sheet1.pictures.add(fig_1, name="Pie Chart", update=True, top=sheet1.range("D38").top, left=sheet1.range("D38").left)
            sheet1.pictures.add(fig_2, name="Risk Contribution", update=True, top=sheet1.range("O38").top, left=sheet1.range("O38").left)
            sheet1.pictures.add(fig_3, name="Bar Plot", update=True, top=sheet1.range("D65").top, left=sheet1.range("D65").left)
            sheet1.pictures.add(fig_4, name="Histogram", update=True, top=sheet1.range("O65").top, left=sheet1.range("O65").left)
            sheet1.pictures.add(fig_5, name="Efficient Frontier", update=True, top=sheet1.range("D9").top, left=sheet1.range("D9").left)

            # Save the Excel file
            wb.save(f"{file_path}.xlsx")
            wb.close()

        logging.info("Plots saved successfully in the Excel file")

    except Exception as e:
        logging.error("Error saving plots to Excel: %s", e)
        return None  # Exit early if saving to Excel fails

    return w  # Return the optimized portfolio weights



In [6]:
# Specific optimization functions

def ulcer_index_optimization(returns_test):
    return portfolio_optimization(returns_test, method_num=3, method_name='Ulcer_Index_Optimization',
                                  model='Classic', opt_method='optimization', rm='UI')

def evar_optimization(returns_test):
    return portfolio_optimization(returns_test, method_num=4, method_name='EVaR_Optimization',
                                  model='Classic', opt_method='optimization', rm='EVaR')

def gmd_optimization(returns_test):
    return portfolio_optimization(returns_test, method_num=5, method_name='GMD_Optimization',
                                  model='Classic', opt_method='optimization', rm='GMD')

def black_litterman_optimization(returns_test, P, Q, Omega):
    return portfolio_optimization(returns_test, method_num=6, method_name='Black_Litterman_Optimization',
                                  model='BL', opt_method='optimization', P=P, Q=Q, Omega=Omega)

def risk_parity_optimization(returns_test):
    return portfolio_optimization(returns_test, method_num=7, method_name='Risk_Parity_Optimization',
                                  model='Classic', opt_method='rp_optimization')

def hrp_optimization(returns_test):
    return portfolio_optimization(returns_test, method_num=8, method_name='HRP_Optimization',
                                  model='HRP', opt_method='optimization', codependence='pearson', linkage='single')

def hierarchical_clustering_optimization(returns_test):
    return portfolio_optimization(returns_test, method_num=9, method_name='HERC_Optimization',
                                  model='HERC', opt_method='optimization', codependence='pearson', linkage='ward')


In [7]:

# Replace 'data' with your actual returns data
data = returns.copy()

# Historical Mean-Risk Optimization
w_mean_risk = portfolio_optimization(data, method_num=2, method_name='Mean_Risk_Optimization',
                                     model='Classic', opt_method='optimization', rm='MV', obj='Sharpe')

2024-10-04 23:39:53,620 - INFO - Running Mean_Risk_Optimization Portfolio Optimization using Classic model and optimization method
2024-10-04 23:39:53,621 - INFO - Portfolio object created successfully
2024-10-04 23:39:53,636 - INFO - Asset stats calculated with method_mu='hist' and method_cov='hist'
2024-10-04 23:39:53,638 - INFO - Optimization method selected: optimization
2024-10-04 23:39:53,770 - INFO - Optimization completed successfully
2024-10-04 23:39:53,771 - INFO - Portfolio weights stored successfully
2024-10-04 23:39:58,426 - INFO - Efficient frontier generated with 50 points
2024-10-04 23:40:03,091 - INFO - Excel report generated successfully at ../reports/05 - portfolio_optimization_riskfolio\02 - Mean_Risk_Optimization
2024-10-04 23:40:04,094 - INFO - Drawdown plot created successfully
2024-10-04 23:40:05,651 - INFO - All plots created successfully
2024-10-04 23:40:14,172 - INFO - Plots saved successfully in the Excel file


In [8]:

# Ulcer Index Optimization
# w_ui = ulcer_index_optimization(data)

In [9]:

# Entropic Value at Risk Optimization
# w_evar = evar_optimization(data)

In [ ]:

# Gini Mean Difference Optimization
w_gmd = gmd_optimization(data)

2024-10-04 23:40:14,207 - INFO - Running GMD_Optimization Portfolio Optimization using Classic model and optimization method
2024-10-04 23:40:14,209 - INFO - Portfolio object created successfully
2024-10-04 23:40:14,213 - INFO - Asset stats calculated with method_mu='hist' and method_cov='hist'
2024-10-04 23:40:14,214 - INFO - Optimization method selected: optimization


In [ ]:

# Black-Litterman Optimization (with dummy inputs for P, Q, Omega)
P = pd.DataFrame(np.random.rand(2, data.shape[1]), columns=data.columns)
Q = pd.Series(np.random.rand(2))
Omega = np.diag(np.random.rand(2))
w_bl = black_litterman_optimization(data, P, Q, Omega)

In [ ]:

# Risk Parity Optimization
w_rp = risk_parity_optimization(data)

In [ ]:

# Hierarchical Risk Parity Optimization
w_hrp = hrp_optimization(data)

In [ ]:

# Hierarchical Clustering Optimization
w_herc = hierarchical_clustering_optimization(data)

In [ ]:

# Print results
print("Mean-Risk Weights: ", w_mean_risk)
print("Ulcer Index Weights: ", w_ui)
print("EVaR Weights: ", w_evar)
print("GMD Weights: ", w_gmd)
print("Black-Litterman Weights: ", w_bl)
print("Risk Parity Weights: ", w_rp)
print("HRP Weights: ", w_hrp)
print("HERC Weights: ", w_herc)

In [ ]:


# 2. Mean-Risk with Custom Estimates
def mean_risk_custom_optimization(returns, custom_mean, custom_cov):
    print("Running Mean-Risk Portfolio Optimization using custom estimates")
    return optimize_portfolio(returns, model='Classic', risk_measure='MV', custom_mean=custom_mean, custom_cov=custom_cov)

# 3. Ulcer Index Portfolio Optimization
def ulcer_index_optimization(returns):
    print("Running Ulcer Index Portfolio Optimization")
    return optimize_portfolio(returns, model='Classic', risk_measure='UI')

# 4. Entropic Value at Risk (EVaR) Optimization
def evar_optimization(returns):
    print("Running EVaR Portfolio Optimization")
    return optimize_portfolio(returns, model='Classic', risk_measure='EVaR')

# 5. Gini Mean Difference (GMD) Optimization
def gmd_optimization(returns):
    print("Running GMD Portfolio Optimization")
    return optimize_portfolio(returns, model='Classic', risk_measure='GMD')

# 6. Black-Litterman Model
def black_litterman_optimization(returns, P, Q, Omega):
    print("Running Black-Litterman Model Portfolio Optimization")
    port = rp.Portfolio(returns=returns)
    port.mu_bl, port.cov_bl = port.black_litterman(P, Q, Omega)
    w = port.optimization(model='BL')
    return w

# 7. Risk Parity Optimization
def risk_parity_optimization(returns):
    print("Running Risk Parity Optimization")
    return optimize_portfolio(returns, model='RiskParity', risk_measure='MV')

# 8. Hierarchical Risk Parity (HRP) Optimization
def hrp_optimization(returns):
    print("Running HRP Portfolio Optimization")
    port = rp.HCPortfolio(returns=returns)
    w = port.optimization(model='HRP', codependence='pearson', linkage='single', risk_measure='MV')
    return w

# 9. Hierarchical Clustering and Networks
def hierarchical_clustering_optimization(returns):
    print("Running Hierarchical Clustering Optimization with Network Constraints")
    port = rp.HCPortfolio(returns=returns)
    w = port.optimization(model='HERC', codependence='pearson', linkage='ward', risk_measure='MV')
    return w

# 10. Real-World Application with MOSEK
def mosek_real_world_optimization(returns):
    print("Running Optimization with MOSEK Solver")
    # MOSEK integration example
    # port.optimization(model='Classic', solver='MOSEK')  # Uncomment if MOSEK is available
    pass

# Main execution flow
if __name__ == "__main__":
    # Example usage for testing (replace with your actual data)

    # Historical Mean-Risk Optimization
    w_mean_risk = mean_risk_optimization(data)

    # Custom Mean-Risk Optimization (Example with random custom estimates)
    #custom_mean = pd.Series(np.random.rand(data.shape[1]), index=data.columns)
    #custom_cov = pd.DataFrame(np.random.rand(data.shape[1], data.shape[1]), index=data.columns, columns=data.columns)
    #w_custom = mean_risk_custom_optimization(data, custom_mean, custom_cov)

    # Ulcer Index Optimization
    w_ui = ulcer_index_optimization(data)

    # Entropic Value at Risk Optimization
    w_evar = evar_optimization(data)

    # Gini Mean Difference Optimization
    w_gmd = gmd_optimization(data)

    # Black-Litterman Example (Dummy inputs for P, Q, Omega)
    P = pd.DataFrame(np.random.rand(2, data.shape[1]), columns=data.columns)
    Q = pd.Series(np.random.rand(2))
    Omega = np.diag(np.random.rand(2))
    w_bl = black_litterman_optimization(data, P, Q, Omega)

    # Risk Parity Optimization
    w_rp = risk_parity_optimization(data)

    # Hierarchical Risk Parity Optimization
    w_hrp = hrp_optimization(data)

    # Hierarchical Clustering Optimization with Network Constraints
    w_herc = hierarchical_clustering_optimization(data)

    # Real-World Application with MOSEK
    mosek_real_world_optimization(data)

    # Print results (for each optimization run)
    print("Mean-Risk Weights: ", w_mean_risk)
    print("Custom Mean-Risk Weights: ", w_custom)
    print("Ulcer Index Weights: ", w_ui)
    print("EVaR Weights: ", w_evar)
    print("GMD Weights: ", w_gmd)
    print("Black-Litterman Weights: ", w_bl)
    print("Risk Parity Weights: ", w_rp)
    print("HRP Weights: ", w_hrp)
    print("HERC Weights: ", w_herc)
